The following is a demo to download a list of ETF tickers from yahoo finance using asyncio.

## Import the modules

In [1]:
import os
import sys
from datetime import datetime as dt
import asyncio
from concurrent.futures import ProcessPoolExecutor
import pickle
import logging

!{sys.executable} -m pip install yfinance
import yfinance

chunking a list of items into n-sized chunks

In [2]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

async fetch code to fetch data using yfinance module

In [3]:
def fetch(tickers, data_dir, period):
    errors = list()
    for ticker in tickers:
        try:
            logging.debug('downloading %s' % ticker)
            stock = yfinance.Ticker(ticker)
            df = stock.history(period=period)
            if not df.empty:
                with open('%s/%s_prices.pkl' % (data_dir, ticker), 'wb') as output:
                    pickle.dump(df, output, pickle.HIGHEST_PROTOCOL)
        except Exception as ex:
            logging.error(ex)
            errors.append(ticker)
            continue
    return errors


async def async_fetch(tickers, data_dir, period):
    result = await loop.run_in_executor(executor, fetch, tickers, data_dir, period)
    return result


def run_async(ticker_lists, data_dir, period):
    tasks = []
    for tickers in ticker_lists:
        tasks.append(loop.create_task(async_fetch(tickers, data_dir, period)))

    done, _ = loop.run_until_complete(asyncio.wait(tasks))
    all_errors = list()
    for fut in done:
        all_errors = all_errors + fut.result()
    print(all_errors)
    

main code to get started

In [4]:
loop = asyncio.get_event_loop()
executor = ProcessPoolExecutor(max_workers=10)
working_dir = os.getcwd()
tickers = ['UVXY', 'SPY', 'SQQQ', 'XLF', 'VXX', 'EEM', 'SLV', 'TZA', 'TQQQ', 'QQQ', 'XLE', 'EWZ', 'IWM',
          'HYG', 'GDX', 'IAU', 'EFA', 'FXI', 'LQD']

start_time = dt.now()

ticker_lists = chunks(tickers, 5)

logging.info("async run start")
run_async(ticker_lists, working_dir, '6mo')
logging.info("async run finish")

end_time = dt.now()

RuntimeError: This event loop is already running